# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Building composable functions”

Abraham warns the reader that this lesson represents a rapid departure from the comfort zone of C#. This is done by identifying several F# equivalents of what we might be used to in C#:

- The F# equivalent of C# method overloading is _currying_.
- The F# equivalent of C# extension methods is _pipelining_ with the forward-pipe operator, `|>`.
- The F# way to express what C# folks might call imperative method chaining is with the compose operator, `>>`.


## Comparing methods and functions

| | C# methods | F# `let`-bound functions |
|- |- |-
| Behavior | Statements of expressions | Expressions by default |
| Scope | Instance [object] or static [type] | Static [module level of nested function] |
| Overloading | Allowed | Not supported |
| Currying | Not supported | Native support |

Just because C# methods do not support currying does not mean it is not supported in the language. `Func` in C# [can be used](https://wensveen.wordpress.com/2010/03/01/currying-in-csharp/).


## Currying and method overloading

In the world of F#, currying is regarded as “[completely unrelated](https://stackoverflow.com/a/10613522/22944)” to method overloading so it is understandable why Abraham would simply state that overloading is not supported in F# in the table above. However, novices like myself feel [compelled to question](https://stackoverflow.com/questions/10610665/is-currying-the-same-as-overloading) this lack of a relationship.

When we see the definition of overloading as:

>…providing multiple definitions for the same name and having the compiler select which definition is used each time the name is used…

then we can rest assured that there is no relationship. But it is easy to get confused when we feel that the _intent_ behind overloading can be very, very similar to the _intent_ behind currying.

In almost every C# design, my intent behind overloading is to define a central method with logic, referenced by multiple methods calling this central method with fewer parameters than this central method:


In [ ]:
static class AddUtility
{
    public static int Add(int a, int b) => a + b; // my ‘central’ method
    public static int Add(int b) => Add(5, b);
}

AddUtility.Add(10) // here the compiler chooses which `Add` to call (nothing to do with currying)

15

I _can_ accept that what we have above is an attempt to implement currying while using (or abusing?) overloading. What F# will do for us all is clarify my intent!


## Partial function application

_Partial application_ is the act of calling a [curried](https://en.wikipedia.org/wiki/Currying) function. _Currying_, by the way, refers to a person, [Haskell Brooks Curry](https://en.wikipedia.org/wiki/Haskell_Curry).

>Partially applied functions are one of the most powerful parts of the function system in F# as compared to C#. These functions open up all sorts of interesting possibilities …the following two functions appear to do the same thing, except that one uses brackets (parentheses) and commas for input arguments (like C#), and one doesn’t. The former is referred to as _tupled_ form and the latter as _curried_ form.


In [ ]:
#!fsharp

let tupledAdd(a,b) = a + b

tupledAdd (5,10)

15

In [ ]:
#!fsharp

let curriedAdd a b = a + b

curriedAdd 5 10

15

>- _Tupled_ functions force you to supply all the arguments at once (like C# methods) and have a signature of `(type1 * type2 … * typeN) -> result`. F# considers all the arguments as a _single object_, which is why the signature looks like a tuple signature—that’s exactly what it is.
>- Curried functions allow you to supply _some_ of the arguments to a function, and get back a _new function_… Curried functions have a signature of `arg1 -> arg2 … -> argN -> result`.


In [ ]:
#!fsharp

let addFive = curriedAdd 5 // `addFive` is a curried function from the partial application of 5

let constrainedResult = addFive 10

constrainedResult

15

By hovering the mouse pointer over the bindings in the code above, we can verify the type signatures that Abraham promises. The `constrainedResult` is simply of type `int`:


In [ ]:
#!fsharp

constrainedResult.GetType()

System.Int32

>One easy use for curried functions is in creating a _more constrained_ version of a function, sometimes known as a _wrapper_ function.

The `constrainedResult` above represents calling a _constrained_ version of `curriedAdd`.

And, by the way, my leading intent behind using the method-overloading compiler feature in C# is to defined _constrained_ versions of a ‘central’ method.


## Pipelines

The intent to obtain `constrainedResult` can be made clearer in a _single_ expression with the forward pipe operator, `|>`:


In [ ]:
#!fsharp

let constrainedResult = 5 |> curriedAdd 10

constrainedResult

15

By inspecting the last time we made a binding with `constrainedResult`, we can see that `5 |> curriedAdd` is the equivalent of `addFive`. Abraham explains what `5 |> curriedAdd` is expressing:

>Take the value on the _left-hand side_ of the pipe, and flip it over to the _right-hand side_ as the _last_ argument to the function.

We can demonstrate the bit about the last argument with this binding:


In [ ]:
#!fsharp

let subtract a b = a - b

5 |> subtract 10

5

In [ ]:
#!fsharp

10 |> subtract 5

-5

These `subtract` expressions are confusing, however. As we read from left to right, we are expecting `5 - 10` and `10 - 5`. One workaround might be this:


In [ ]:
#!fsharp

let subtract a b = -a + b

5 |> subtract 10

-5

In [ ]:
#!fsharp

10 |> subtract 5

5

## Extension methods vs. curried functions



Abraham leads us to suppose that something like `10 |> subtract 5` looks like `10.Subtract(5)` in C#, where `Subtract` would be an [extension method](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/extension-methods) of `int`:

>This might look similar to a feature that already exists in C# and VB: extension methods. But they’re not quite the same…

| | C# extension methods | F# |
|- |- |-
| Scope | Methods must be explicitly designed to be extension methods in a static class with the extension point decorated with the `this` keyword. | Any single-argument .NET method (including the BCL) and all curried functions can be chained together. |
| Extension point | First argument in method signature. | Last argument in function. |
| Currying support | None | First class. |
| Paradigm | Not always a natural fit for OO paradigm with private state. | Natural fit for stateless functions. |


## Pipelining the `drive` function from lesson 6

Recall the `drive` function we composed in [lesson 6](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/06-working-with-immutable-data.ipynb):


In [ ]:
#!fsharp

let drive(petrol, distance) =
    if distance = "far" then petrol / 2.0
    elif distance = "medium" then petrol - 10.0
    else petrol - 1.0

In order for pipelining to work, reading left to right, we have to flip the function arguments and use curried syntax:


In [ ]:
#!fsharp

let drive distance petrol =
    if distance = "far" then petrol / 2.0
    elif distance = "medium" then petrol - 10.0
    else petrol - 1.0


This new version of `drive`, shadowing the former, should work exactly like the former:


In [ ]:
#!fsharp

let petrol = 100.0

let firstState = drive "far" petrol
let secondState = drive "medium" firstState
let finalState = drive "short" secondState

finalState

39

Now that we have the `drive` function arguments in currying order, let us get rid of all those `*State` bindings above and use elegant pipelining:


In [ ]:
#!fsharp

petrol
|> drive "far"
|> drive "medium"
|> drive "short"

39

>You’ll find that the pipeline is extremely useful for composing code together into a human-readable [domain-specific language](https://en.wikipedia.org/wiki/Domain-specific_language) (DSL).


## Composing functions together

>The last element to touch upon in this lesson is the somewhat less common _compose_ operator (`>>`)…

With the compose operator, we can bind several functions to one function:


In [ ]:
#!fsharp

let goOnMyDrive = drive "far" >> drive "medium" >> drive "short"

goOnMyDrive petrol

39

The compose operator is combining curried functions into one function, `goOnMyDrive`.


@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.245901+0512fe0a05f5a99419a543463ee68233a3c60371Build date: 2021-09-10T01:15:17.7422137Zhttps://github.com/dotnet/interactive
